In [1]:
from AHI_AC import *

# Set spatial resolution
# 0.01deg ==> 0.01, 0.02deg ==> 0.02
res = 0.01

# Set latitude upper and bottom bounds
lat_up, lat_bottom = 60, -60

# Set longitude left and right bounds
lon_left, lon_right = 85, 205

# Set save folder
target = '/data01/GEO/SRDATA/'

# See main.py
row,col,row_AHI,col_AHI,AHI_lat,AHI_lon,AHI_VZA, AHI_VAA,AHI_AL,Water_idx,Landmask = main_read(res,lat_up, lat_bottom,lon_left, lon_right)

# Read LUT and interpolate with LUT
# Processing band can be set by yourself!
interpolation = LUT_interpolator(1)
fn1_1, fn2_1, fn3_1 = interpolation._interpolate()
interpolation = LUT_interpolator(2)
fn1_2, fn2_2, fn3_2 = interpolation._interpolate()
interpolation = LUT_interpolator(3)
fn1_3, fn2_3, fn3_3 = interpolation._interpolate()
interpolation = LUT_interpolator(4)
fn1_4, fn2_4, fn3_4 = interpolation._interpolate()

# Set necessary parameters
band_type = {'1': 'vis', '2': 'vis', '3': 'ext', '4': 'vis'}
band_num = {'1': '01', '2': '02', '3': '01', '4': '03'}
calculate_6s = {'1': calculate_6s_band1, '2': calculate_6s_band2, '3': calculate_6s_band3, '4': calculate_6s_band4}
fn1 = {'1': fn1_1, '2': fn1_2, '3': fn1_3, '4': fn1_4}
fn2 = {'1': fn2_1, '2': fn2_2, '3': fn2_3, '4': fn2_4}
fn3 = {'1': fn3_1, '2': fn3_2, '3': fn3_3, '4': fn3_4}


In [2]:
def SR_Process(band):
    start_time = T.time()

    # Download AHI from NAS
    print(f"Start download AHI for Band{band}")
    S_Read_AHI_time = T.time()
    num_threads = 4  # 

    # No decompression
    H8 = H8_data('liwei', 'liwei00', band_type[band], band_num[band], date, folder_original)
    NAS_file_path,Target_path = H8.file_path(),H8.target_path()
    multi_threaded_copy(NAS_file_path, Target_path, num_threads)
    decompression(Target_path)    
    AHI_data = H8.read_H8data(Target_path[:-4])

    # If there is no data, skip
    if AHI_data == 'No data':
        print(f"No AHI data for Band{band}, pass")
        return None
    
    # Cosine correction
    AHI_data = AHI_data / np.cos(np.radians(AHI_SZA))
    E_Read_AHI_time = T.time()
    print(f"Finish download AHI for Band{band}")
    print('AHI ORG data processing cost : {}s'.format(round((E_Read_AHI_time-S_Read_AHI_time),4)))
    
    # SR calculate
    print(f'Start Calculate SR for Band{band}')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=njob)(delayed(calculate_6s[band])(fn1[band], fn2[band], fn3[band], Landmask, Aerosol_type, *params[band], AHI_data, i) for i in range(row_AHI))
    SR = np.full((row_AHI,col_AHI), np.nan)
    
    # Refill the result into land area
    for i in range(row_AHI):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print(f'Calculate SR for Band{band} finished')
    print('Calculate SR cost : {}s'.format(round((E_SR_time-S_SR_time),4)))

    # Conver file to int16 and fill nan with -32768
    print(f'Start Save SR file for Band{band}')
    S_SAVE_time = T.time()
    SR = np.array(SR).reshape(row_AHI,col_AHI)
    SR = np.around(SR, 4) * 10000
    SR = np.clip(SR, -32767, 32767)
    SR = np.where(np.isnan(SR), -32768, SR)
    
    # Save SR file
    SR_file = open(folder_AC + f'H08_{YYYY}{MM}{DD}_{HH}{MIN}_B0{band}_SR.dat', 'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    E_SAVE_time = T.time()
    print(f'Save Band{band} SR finished')
    print('Save file cost : {}s'.format(round((E_SAVE_time-S_SAVE_time),4)))
    
    # if band == '4':
    #     remove_original_file(folder_original)
    #     print("delete file finish")
    
    end_time = T.time()
    TIME = end_time - start_time
    print('Band{} cost time: {:} secs, {:.1f} mins\n'.format(band, TIME, TIME / 60))

In [ ]:
error_list = []

njob = 30
date_start = '2020-01-08 13:00'
date_end = '2020-01-31 23:50'
date_t = dt.timedelta(minutes=10)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
# print("Start {} ".format(date_start[:-6]))
YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
while date_time_now <= date_e:
    try:
        date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
        YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)   
        hour = int(date[8:10])
        print(date_time_now)
        if  hour >= 14 and hour <= 22:
        # if  hour >= 8:
            print(f"{date_time_now} not in time range, pass")
        else:
            S_time = T.time()
            print("Start processing {}".format(date))
            
            # make dir for original and AC data
            folder_original = f"{target}{date}_original/"
            TOA_PATH = f'/data01/GEO/SRDATA_LAI/{date}_original/'
            folder_AC = f"{target}{YYYY}/{MM}/{DD}/"
            mkdir(folder_original)
            mkdir(folder_AC)

            # Read Solar angle
            print('Start read solar angle')
            S_ANGLE_time = T.time()
            AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
            AHI_SZA,AHI_SAA = AHI_SZA[row:row+row_AHI,col:col+col_AHI],AHI_SAA[row:row+row_AHI,col:col+col_AHI]
            RAA = abs(AHI_SAA - AHI_VAA)
            RAA[RAA>180]=360-RAA[RAA>180]
            print('Finish read solar angle')
            E_ANGLE_time = T.time()
            print('Read angle cost : {}s'.format(round((E_ANGLE_time-S_ANGLE_time),4)))

            # Atmosphereic data
            print('Start read CAMS')
            S_ATMOS_time = T.time()
            CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
            Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
            print('Finish read CAMS')
            E_ATMOS_time = T.time()
            print('Read Atmospheric data cost : {}s'.format(round((E_ATMOS_time-S_ATMOS_time),4)))

            params = {
                        '1': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                        '2': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                        '3': [WV, OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                        '4': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
                    }

            for band in ['1','2','3', '4']:
                SR_Process(band)
            E_time = T.time()
            print('Total time cost : {}s'.format(round((E_time-S_time),4)))
        # else:
            # print(f"{date_time_now} not in time range, pass")
    except:
        error_list.append(date)
    date_time_now = date_time_now + date_t

# WITH INPUT DATE

In [13]:
date_list = ['201808010300',
 '201808020030',
 '201808020200',
 '201808020340',
 '201808020350',
 '201808020410',
 '201808020420',
 '201808020430',
 '201808020440',
 '201808020450',
 '201808030250',
 '201808040230',
 '201808040250',
 '201808040300',
 '201808040310',
 '201808040320',
 '201808040330',
 '201808050230',
 '201808050410',
 '201808060140',
 '201808060310',
 '201808060320',
 '201808070220',
 '201808080300',
 '201808080310',
 '201808090210',
 '201808090340',
 '201808090350',
 '201808100250',
 '201808110020',
 '201808110200',
 '201808110230',
 '201808110300',
 '201808110310',
 '201808110320',
 '201808110330',
 '201808110340',
 '201808110350',
 '201808110400',
 '201808110410',
 '201808110420',
 '201808110430',
 '201808130140',
 '201808130150',
 '201808130250',
 '201808130300',
 '201808130310',
 '201808130320',
 '201808130330',
 '201808130340',
 '201808130350',
 '201808130400',
 '201808130410',
 '201808130420',
 '201808130430',
 '201808140230',
 '201808150230',
 '201808150250',
 '201808150300',
 '201808150310',
 '201808150320',
 '201808150330',
 '201808160040',
 '201808160210',
 '201808160220',
 '201808170250',
 '201808170300',
 '201808180030',
 '201808180200',
 '201808180340',
 '201808190250',
 '201808200320',
 '201808200330',
 '201808200340',
 '201808200350',
 '201808200400',
 '201808200410',
 '201808200420',
 '201808200430',
 '201808210230',
 '201808210410',
 '201808220140',
 '201808220310',
 '201808220320',
 '201808220330',
 '201808220340',
 '201808220350',
 '201808220400',
 '201808220410',
 '201808220420',
 '201808220430',
 '201808230220',
 '201808240300',
 '201808240310',
 '201808240320',
 '201808240330',
 '201808240340',
 '201808240350',
 '201808240400',
 '201808240410',
 '201808240420',
 '201808240430',
 '201808250210',
 '201808250340',
 '201808250350',
 '201808260250',
 '201808270020',
 '201808270200',
 '201808270330',
 '201808270340',
 '201808290140',
 '201808290150',
 '201808290320',
 '201808300220',
 '201808300230',
 '201808310310',
 '201808310320']

In [14]:
njob = 30

for DATE in date_list:
    
    YYYY,MM,DD,HH,MIN,date= DATE[0:4],DATE[4:6],DATE[6:8],DATE[8:10],DATE[10:12],DATE
    print(DATE)
    # if  hour >=9 and hour <= 22:

    S_time = T.time()
    print("Start processing {}".format(date))

    # make dir
    folder_original = f"{target}{date}_original/"
    TOA_PATH = f'/data01/GEO/SRDATA_LAI/{date}_original/'
    folder_AC = f"{target}{YYYY}/{MM}/{DD}/"
    mkdir(folder_original)
    mkdir(folder_AC)

    # Solar angle
    print('Start read solar angle')
    S_ANGLE_time = T.time()
    AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
    AHI_SZA,AHI_SAA = AHI_SZA[row:row+row_AHI,col:col+col_AHI],AHI_SAA[row:row+row_AHI,col:col+col_AHI]
    RAA = abs(AHI_SAA - AHI_VAA)
    RAA[RAA>180]=360-RAA[RAA>180]
    print('Finish read solar angle')
    E_ANGLE_time = T.time()
    print('Read angle cost : {}s'.format(round((E_ANGLE_time-S_ANGLE_time),4)))

    # Atmosphereic data
    print('Start read CAMS')
    S_ATMOS_time = T.time()
    CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
    Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
    print('Finish read CAMS')
    E_ATMOS_time = T.time()
    print('Read Atmospheric data cost : {}s'.format(round((E_ATMOS_time-S_ATMOS_time),4)))

    params = {
                '1': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                '2': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                '3': [WV, OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                '4': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
            }

    for band in ['1', '2', '3', '4']:
        SR_Process(band)
    E_time = T.time()
    print('Total time cost : {}s'.format(round((E_time-S_time),4)))
    # else:
        # print(f"{date_time_now} not in time range, pass")


201808010300
Start processing 201808010300
Start read solar angle
Finish read solar angle
Read angle cost : 6.195s
Start read CAMS
Finish read CAMS
Read Atmospheric data cost : 130.5653s
Start download AHI for Band1


/tmp/ipykernel_3667642/3398090805.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Finish download AHI for Band1
AHI ORG data processing cost : 2.7671s
Start Calculate SR for Band1
Calculate SR for Band1 finished
Calculate SR cost : 56.5779s
Start Save SR file for Band1
Save Band1 SR finished
Save file cost : 10.9922s
Band1 cost time: 70.33792185783386 secs, 1.2 mins

Start download AHI for Band2
Finish download AHI for Band2
AHI ORG data processing cost : 3.8892s
Start Calculate SR for Band2
Calculate SR for Band2 finished
Calculate SR cost : 27.8313s
Start Save SR file for Band2
Save Band2 SR finished
Save file cost : 6.0439s
Band2 cost time: 37.76491189002991 secs, 0.6 mins

Start download AHI for Band3
Finish download AHI for Band3
AHI ORG data processing cost : 34.075s
Start Calculate SR for Band3
Calculate SR for Band3 finished
Calculate SR cost : 52.444s
Start Save SR file for Band3
Save Band3 SR finished
Save file cost : 7.727s
Band3 cost time: 94.24677109718323 secs, 1.6 mins

Start download AHI for Band4
Finish download AHI for Band4
AHI ORG data processing

/tmp/ipykernel_3667642/3398090805.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Finish download AHI for Band1
AHI ORG data processing cost : 2.3141s
Start Calculate SR for Band1
Calculate SR for Band1 finished
Calculate SR cost : 27.1806s
Start Save SR file for Band1
Save Band1 SR finished
Save file cost : 6.1382s
Band1 cost time: 35.63346886634827 secs, 0.6 mins

Start download AHI for Band2
Finish download AHI for Band2
AHI ORG data processing cost : 2.0746s
Start Calculate SR for Band2
Calculate SR for Band2 finished
Calculate SR cost : 27.5827s
Start Save SR file for Band2
Save Band2 SR finished
Save file cost : 10.4468s
Band2 cost time: 40.104615211486816 secs, 0.7 mins

Start download AHI for Band3
Finish download AHI for Band3
AHI ORG data processing cost : 31.7514s
Start Calculate SR for Band3
Calculate SR for Band3 finished
Calculate SR cost : 53.7456s
Start Save SR file for Band3
Save Band3 SR finished
Save file cost : 9.4774s
Band3 cost time: 94.97503542900085 secs, 1.6 mins

Start download AHI for Band4
Finish download AHI for Band4
AHI ORG data proces

...

Finish download AHI for Band1
AHI ORG data processing cost : 3.1458s
Start Calculate SR for Band1
Calculate SR for Band1 finished
Calculate SR cost : 29.5326s
Start Save SR file for Band1
Save Band1 SR finished
Save file cost : 14.8833s
Band1 cost time: 47.56225061416626 secs, 0.8 mins

Start download AHI for Band2
Finish download AHI for Band2
AHI ORG data processing cost : 3.9713s
Start Calculate SR for Band2
Calculate SR for Band2 finished
Calculate SR cost : 30.8711s
Start Save SR file for Band2
Save Band2 SR finished
Save file cost : 13.4613s
Band2 cost time: 48.30423951148987 secs, 0.8 mins

Start download AHI for Band3
Finish download AHI for Band3
AHI ORG data processing cost : 34.6458s
Start Calculate SR for Band3
Calculate SR for Band3 finished
Calculate SR cost : 57.4396s
Start Save SR file for Band3
Save Band3 SR finished
Save file cost : 19.0806s
Band3 cost time: 111.1667869091034 secs, 1.9 mins

Start download AHI for Band4
Finish download AHI for Band4
AHI ORG data proce

/tmp/ipykernel_3667642/3398090805.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Finish download AHI for Band1
AHI ORG data processing cost : 3.3187s
Start Calculate SR for Band1
Calculate SR for Band1 finished
Calculate SR cost : 30.4083s
Start Save SR file for Band1
Save Band1 SR finished
Save file cost : 14.9952s
Band1 cost time: 48.72275972366333 secs, 0.8 mins

Start download AHI for Band2
Finish download AHI for Band2
AHI ORG data processing cost : 6.1292s
Start Calculate SR for Band2
Calculate SR for Band2 finished
Calculate SR cost : 31.4386s
Start Save SR file for Band2
Save Band2 SR finished
Save file cost : 13.9064s
Band2 cost time: 51.47490572929382 secs, 0.9 mins

Start download AHI for Band3
Finish download AHI for Band3
AHI ORG data processing cost : 39.5257s
Start Calculate SR for Band3
Calculate SR for Band3 finished
Calculate SR cost : 56.9404s
Start Save SR file for Band3
Save Band3 SR finished
Save file cost : 17.0069s
Band3 cost time: 113.47382402420044 secs, 1.9 mins

Start download AHI for Band4
Finish download AHI for Band4
AHI ORG data proc

/tmp/ipykernel_3667642/3398090805.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Finish download AHI for Band1
AHI ORG data processing cost : 3.946s
Start Calculate SR for Band1
Calculate SR for Band1 finished
Calculate SR cost : 30.5913s
Start Save SR file for Band1
Save Band1 SR finished
Save file cost : 13.8459s
Band1 cost time: 48.383784532547 secs, 0.8 mins

Start download AHI for Band2
Finish download AHI for Band2
AHI ORG data processing cost : 6.3564s
Start Calculate SR for Band2
Calculate SR for Band2 finished
Calculate SR cost : 31.7093s
Start Save SR file for Band2
Save Band2 SR finished
Save file cost : 13.4943s
Band2 cost time: 51.560675621032715 secs, 0.9 mins

Start download AHI for Band3
Finish download AHI for Band3
AHI ORG data processing cost : 39.2747s
Start Calculate SR for Band3
Calculate SR for Band3 finished
Calculate SR cost : 57.9821s
Start Save SR file for Band3
Save Band3 SR finished
Save file cost : 16.2472s
Band3 cost time: 113.50489497184753 secs, 1.9 mins

Start download AHI for Band4
Finish download AHI for Band4
AHI ORG data proces